1. Intensity Based filtering
2. DBSCAN clustering
3. Shape Fitting
4. Geojson Creation

In [21]:
import alphashape
from scipy.stats import gaussian_kde
from pyntcloud import PyntCloud
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from plyfile import PlyData
import plotly.graph_objects as go
import open3d as o3d
import pandas as pd
from shapely.geometry import Polygon, Point
from tqdm import tqdm
import utm
import json

In [22]:
ground_map = '/home/maanz/Downloads/DGV/LidarPipeline_runs/hamburg/maps/ground_map.ply'
offset_file = '/home/maanz/Downloads/DGV/LidarPipeline_runs/hamburg/trajectory_transformed/conversion_offset.json'


In [23]:
with open(offset_file, 'r') as file:
    offset = json.load(file)
print(offset)

{'utm_coords': [387118.53436412767, 5815549.486782843], 'zone_number': 33, 'zone_letter': 'U'}


In [24]:
def read_point_cloud(file_or_cloud):
    if isinstance(file_or_cloud, PyntCloud):
        return file_or_cloud
    elif isinstance(file_or_cloud, str):
        cloud = PyntCloud.from_file(file_or_cloud)
        return cloud
    else:
        raise ValueError("Input must be a file path (str) or a Point Cloud object.")

In [25]:
cloud = read_point_cloud(ground_map)
points = cloud.points.values
intensity = points[:, 3]
print(len(intensity))

3316253


In [26]:
def pyntcloud_to_open3d(file_or_cloud):
    cloud = read_point_cloud(file_or_cloud)
    cloud_pts = cloud.points[['x', 'y', 'z', 'intensity']].values
    
    xyz = cloud_pts[:, :3]
    open3d_cloud = o3d.geometry.PointCloud()
    open3d_cloud.points = o3d.utility.Vector3dVector(xyz)
    return open3d_cloud

In [27]:
def color_pcd(pcd, clr='r'):
    if clr == 'r':
        color = [1, 0, 0]
        pcd.paint_uniform_color(color)
    elif clr == 'g':
        color = [0, 1, 0]
        pcd.paint_uniform_color(color)
    elif clr == 'b':
        color = [0, 0, 1]
        pcd.paint_uniform_color(color)
        
    return pcd

In [28]:
def visualize_multiple_pcds(*pcds):
    clouds = [pcd for pcd in pcds]
    o3d.visualization.draw_geometries(clouds)

In [29]:
def intensity_stats(intensity):
    intensity = intensity.flatten()
    min_intensity, max_intensity = np.min(intensity), np.max(intensity)
    mean_intensity = np.mean(intensity)
    
    std_dev = np.std(intensity)
    return min_intensity, max_intensity, mean_intensity, std_dev

In [30]:
min_intensity, max_intensity, mean_intensity, std_dev = intensity_stats(intensity)
print(f"Mean: {mean_intensity}")
print(f"Std Dev: {std_dev}")
print(f"Min Intensity: {min_intensity}")
print(f"Max Intensity: {max_intensity}")


Mean: 7.32788610458374
Std Dev: 4.9103827476501465
Min Intensity: 1.0
Max Intensity: 255.0


In [140]:
def plot_intensity_histogram(intensity, output_html):
    
    min_intensity, max_intensity, mean_intensity, std_dev = intensity_stats(intensity)
    filter_lower_bound, filter_upper_bound = intensity_filter(intensity, 5)
    # Calculate KDE
    kde = gaussian_kde(intensity)
    intensity_values = np.linspace(min_intensity, max_intensity, 1000)
    density = kde(intensity_values)
    
    # Create the histogram
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=intensity,
        nbinsx=80,
        histnorm='probability density',
        name='Histogram',
        opacity=1
    ))
    
    # Add KDE line
    fig.add_trace(go.Scatter(
        x=intensity_values,
        y=density,
        mode='lines',
        name='KDE',
        line=dict(color='red')
    ))
    
    # Add mean and standard deviation lines
    fig.add_trace(go.Scatter(
        x=[mean_intensity, mean_intensity],
        y=[0, max(density)],
        mode='lines',
        name='Mean',
        line=dict(color='red', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=[mean_intensity + std_dev, mean_intensity + std_dev],
        y=[0, max(density)],
        mode='lines',
        name='+1 std',
        line=dict(color='green', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=[mean_intensity - std_dev, mean_intensity - std_dev],
        y=[0, max(density)],
        mode='lines',
        name='-1 std',
        line=dict(color='green', dash='dash')
    ))
    # Add shaded region for intensity filter
    fig.add_trace(go.Scatter(
        x=[filter_lower_bound, filter_lower_bound, filter_upper_bound, filter_upper_bound],
        y=[0, max(density), max(density), 0],
        fill='toself',
        fillcolor='orange',
        opacity=0.3,
        line=dict(color='orange'),
        name='Intensity Filter'
    ))
    # Update layout
    fig.update_layout(
        title='Histogram, KDE, and Standard Deviation of Intensity',
        xaxis_title='Intensity',
        yaxis_title='Density / Frequency',
        xaxis=dict(tickmode='linear', dtick=10),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    
    # Save the plot as an HTML file
    fig.write_html(output_html)
    print(f"Histogram saved as {output_html}")

In [141]:
histogram_plot = '/home/ty/Downloads/AAI/test/intensity_histogram.html'

In [142]:
plot_intensity_histogram(intensity, histogram_plot)

Histogram saved as /home/ty/Downloads/AAI/test/intensity_histogram.html


In [31]:
def intensity_filter(intensity, num_std):
    _, _, mean_intensity, std_dev_intensity = intensity_stats(intensity)
    lower_bound = int(mean_intensity + std_dev_intensity)
    upper_bound = int(mean_intensity + (num_std * std_dev_intensity))
    return lower_bound, upper_bound

In [32]:
lower_bound, upper_bound = intensity_filter(intensity, 5)
print(f"Lower Bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")

Lower Bound: 12
Upper bound: 31


In [36]:
def apply_intensity_filter(map_file, filter_range):
    lower, upper = filter_range
    map_cloud = read_point_cloud(map_file)
    map_df = map_cloud.points
    map_points = map_df.values
    
    intensity_axis = map_points[:, 3]
    intensity_mask = np.logical_and(intensity_axis >= lower, intensity_axis <= upper)
    
    inlier_points_df = map_df[intensity_mask]
    outlier_points_df = map_df[~intensity_mask]
    
    # Convert filtered DataFrames back to PyntClouds
    inlier_cloud = PyntCloud(inlier_points_df)
    outlier_cloud = PyntCloud(outlier_points_df)
    
    return inlier_cloud, outlier_cloud
    

In [37]:
inlier_cloud, outlier_cloud = apply_intensity_filter(map_file=ground_map, 
                                                    filter_range=[lower_bound, upper_bound])

In [38]:
inlier_pcd = pyntcloud_to_open3d(inlier_cloud)
outlier_pcd = pyntcloud_to_open3d(outlier_cloud)

inlier_pcd = color_pcd(inlier_pcd, 'r')

In [39]:
visualize_multiple_pcds(inlier_pcd)

In [40]:
def apply_clustering(obj, eps, min_points, print_progress=True):
    cloud = read_point_cloud(obj)
    pcd = pyntcloud_to_open3d(obj)
    cloud_df = cloud.points
    
    # Apply DBSCAN clustering
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=print_progress))
    
    # Add labels to the DataFrame
    cloud_df['cluster'] = labels
    
    clusters = []
    noise_df = cloud_df[cloud_df['cluster'] == -1].copy()  # Noise points
    noise_df.drop(columns=['cluster'], inplace=True)
    noise_cloud = PyntCloud(noise_df)
    
    for label in np.unique(labels):
        if label != -1:  # Skip noise points
            mask = cloud_df['cluster'] == label
            cluster_df = cloud_df[mask].copy()
            cluster_df.drop(columns=['cluster'], inplace=True)
            clusters.append(cluster_df)
    
    # Concatenate all clusters into a single DataFrame
    clusters_df = pd.concat(clusters, ignore_index=True)
    clusters_cloud = PyntCloud(clusters_df)
    
    return clusters, clusters_cloud, noise_cloud
    

In [49]:
clusters_list, clusters, noise = apply_clustering(inlier_cloud, 1, 10, False)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 104


In [50]:
clustered_markings = '/home/maanz/Downloads/DGV/LidarPipeline_runs/clustered_markings.ply'
clusters.to_file(clustered_markings)
print(f"Clustered markings saved at: {clustered_markings}")

Clustered markings saved at: /home/maanz/Downloads/DGV/LidarPipeline_runs/hamburg/clustered_markings.ply


In [51]:
inlier_pcd = pyntcloud_to_open3d(clusters)
outlier_pcd = pyntcloud_to_open3d(noise)

outlier_pcd = color_pcd(outlier_pcd, 'r')

In [52]:
visualize_multiple_pcds(inlier_pcd, outlier_pcd)

In [53]:
test_cluster = PyntCloud(clusters_list[28])
test_cluster_pcd = pyntcloud_to_open3d(test_cluster)
visualize_multiple_pcds(test_cluster_pcd)

In [54]:
def compute_hulls(clusters_list, alpha=5):
    hulls = []
    for cluster in tqdm(clusters_list, desc="Computing Hulls...", total=len(clusters_list)):
        cloud_obj = PyntCloud(cluster)
        cloud_points = cloud_obj.points.values
        xy_pts = cloud_points[:, :2]
        points = [Point(p[0], p[1]) for p in xy_pts] 
        points_tuples = [(p.x, p.y) for p in points]
        try:
            alpha_shape = alphashape.alphashape(points_tuples, alpha)

            if alpha_shape.geom_type == 'Polygon':
                boundary_points = list(alpha_shape.exterior.coords)
                hulls.append(boundary_points)
            else:
                for geom in alpha_shape.geoms:
                    boundary_points = list(geom.exterior.coords)
                    hulls.append(boundary_points)
        except Exception as e:
            print(f"Error computing alpha shape: {e}")
            continue
        
    return hulls 
    

In [55]:
hulls = compute_hulls(clusters_list, 3)

Computing Hulls...: 100%|█████████████████████| 104/104 [00:46<00:00,  2.26it/s]


In [56]:
def convert_to_latlon(hulls_list, offset):

    latlon_hulls = []
    offset_arr, zone_num, zone_letter = offset['utm_coords'], offset['zone_number'], offset['zone_letter']
    for hull in hulls_list:
        latlon = []
        for point in hull:
            x, y = point
            utm_local_x = x + offset_arr[0]
            utm_local_y = y + offset_arr[1]
            lat, lon = utm.to_latlon(utm_local_x, utm_local_y, zone_num, zone_letter)
            latlon.append([lon, lat])
            latlon_matrix = np.array(latlon)
        latlon_hulls.append(latlon_matrix)

    return latlon_hulls

In [57]:
latlon_hulls = convert_to_latlon(hulls, offset)

In [58]:
def extract_geojson(latlon_hulls, output_file):
    features = []
    for i, hull in enumerate(latlon_hulls):
        closed_hull = np.concatenate((hull, [hull[0]]))
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [closed_hull.tolist()]
            },
            "properties": {"polygon": i+1}
        }
        features.append(feature)

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }
    
    with open(output_file, 'w') as file:
        json.dump(geojson, file)

    print(f"GeoJSON data written to {output_file}")

In [59]:
geojson_path = '/home/maanz/Downloads/DGV/LidarPipeline_runs/lane_markings_geojson.json'

In [60]:
extract_geojson(latlon_hulls, geojson_path)

GeoJSON data written to /home/maanz/Downloads/DGV/LidarPipeline_runs/lane_markings_geojson.json
